In [1]:
import gc
import pathlib

import numpy as np
import pandas as pd
from statsmodels.stats.multitest import multipletests

In [2]:
cell_type = "PBMC"

In [3]:
final_df = pd.DataFrame(
    columns=[
        "group1",
        "group2",
        "meandiff",
        "lower",
        "upper",
        "p-adj",
        "reject",
        "features",
    ]
)

In [4]:
# directory to get files from
data_dir = pathlib.Path(f"../results/{cell_type}/").resolve(strict=True)

# save directory
output_file_path = pathlib.Path(f"../results/{cell_type}_combined.parquet")

# get list of files in directory
file_list = [x for x in data_dir.iterdir() if x.is_file()]

list_of_dfs = []
# loop through files
for file in file_list:
    tmp_df = pd.read_parquet(file)
    list_of_dfs.append(tmp_df)
final_df = pd.concat(list_of_dfs, ignore_index=True)

final_df.shape

(7224, 11)

In [5]:
# correct for multiple testing
final_df["reject"], final_df["p-adj_fdr_bh"], _, _ = multipletests(
    final_df["p-adj"], alpha=0.05, method="fdr_bh"
)
final_df.head()

,group1,group2,meandiff,lower,upper,p-adj,reject,features,p-adj_abs,pos_neg,shuffled,p-adj_fdr_bh
0,healthy,pyroptosis,0.2694,0.0000,0.2605,0.2784,False,Cytoplasm_RadialDistribution_ZernikeMagnitude_...,0.2784,positive,True,0.293343
1,apoptosis,pyroptosis,0.2335,0.0000,0.2136,0.2535,False,Cytoplasm_RadialDistribution_ZernikeMagnitude_...,0.2535,positive,True,0.269783
2,apoptosis,healthy,-0.0359,0.0000,-0.0557,-0.0161,True,Cytoplasm_RadialDistribution_ZernikeMagnitude_...,0.0161,negative,True,-0.095022
3,apoptosis,healthy,0.0131,0.0238,-0.0048,0.0309,False,Cytoplasm_AreaShape_Zernike_5_3,0.0309,positive,False,0.052523
4,apoptosis,pyroptosis,0.0009,0.9838,-0.0172,0.0189,True,Cytoplasm_AreaShape_Zernike_5_3,0.0189,positive,False,0.041550


In [6]:
# save the final_df
final_df.to_parquet(output_file_path)